In [2]:
!pip install pymongo

In [1]:
#1. Desarrollar un script en python que baje desde Buenos Aires Data el JSON correspondiente a los Centros Médicos Barriales.
import requests
from pprint import pprint
from pymongo import MongoClient

In [2]:
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/centros-medicos-barriales/centros-medicos-barriales.geojson'
response = requests.get(url)
objeto = response.json()
#pprint(objeto)
pprint(objeto['features'])
for estacion in objeto['features']:
    pprint(estacion['properties'])

[{'geometry': {'coordinates': [-58.48298690680926, -34.62215646653463],
               'type': 'Point'},
  'properties': {'AREA_PROG': 'ALVAREZ',
                 'BARRIO': 'Villa Santa Rita',
                 'COMUNA': '11',
                 'DOMICILIO': '1022 LAMARCA EMILIO',
                 'Dom_geo': 'LAMARCA EMILIO 1022',
                 'Dom_mapa': 'Lamarca Emilio 1022',
                 'ESPECIALID': 'Pediatría, Clínica médica, Tocoginecología.',
                 'NOMBRE': 'CMB N° 2',
                 'OBJECTID': 1,
                 'OBJETO': 'CENTRO MÉDICO BARRIAL',
                 'PISO_DTO': None,
                 'REGION_SAN': 'II',
                 'TELEFONO': '4672-0744'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-58.45255425526388, -34.635635287689155],
               'type': 'Point'},
  'properties': {'AREA_PROG': 'PIÑERO',
                 'BARRIO': 'Flores',
                 'COMUNA': '07',
                 'DOMICILIO': '801 CARABOBO AV.',
              

In [3]:
#2. Insertar ese JSON en una base de datos de mongo.
cliente = MongoClient("mongodb://192.168.0.13:27017/")
bd = cliente['salud']
coleccion = bd['centros']


In [4]:
#coleccion.remove()
coleccion.insert_many(objeto['features'])

In [6]:
#3. Realizar un query que traiga los datos de los Centros Médicos Barrialess correspondientes al barrio de Barracas, ordenados por el campo OBJECTID, descendente.
client = MongoClient('mongodb://192.168.0.13:27017/')
filter={'properties.BARRIO': 'Barracas'}
sort=list({'properties.OBJECTID': -1}.items())
result = client['salud']['centros'].find(filter=filter,sort=sort)
for resultado in result:
   pprint(resultado)

{'_id': ObjectId('63288e1c9d088c57a8f8bd2e'),
 'geometry': {'coordinates': [-58.371794518670065, -34.64618904038591],
              'type': 'Point'},
 'properties': {'AREA_PROG': 'ARGERICH',
                'BARRIO': 'Barracas',
                'COMUNA': '04',
                'DOMICILIO': '1492 ISABEL LA CATOLICA',
                'Dom_geo': 'ISABEL LA CATOLICA 1492',
                'Dom_mapa': 'Isabel la Católica 1492',
                'ESPECIALID': 'Pediatría, Clínica médica, Tocoginecología.',
                'NOMBRE': 'CMB N° 37',
                'OBJECTID': 29,
                'OBJETO': 'CENTRO MÉDICO BARRIAL',
                'PISO_DTO': None,
                'REGION_SAN': 'I',
                'TELEFONO': '4302-6078\r\n'},
 'type': 'Feature'}
{'_id': ObjectId('63288e1c9d088c57a8f8bd29'),
 'geometry': {'coordinates': [-58.37052338074424, -34.633504174073295],
              'type': 'Point'},
 'properties': {'AREA_PROG': 'ARGERICH',
                'BARRIO': 'Barracas',
           

In [7]:
#4. Realizar un query que traiga los datos de las Centros Médicos Barriales donde dentro de las especialidades se incluya Odontología.
import re
client = MongoClient('mongodb://192.168.0.13:27017/')
filter={'properties.ESPECIALID': re.compile(r"Odonto")}
result = client['salud']['centros'].find(filter=filter)
for resultado in result:
   pprint(resultado)

{'_id': ObjectId('63288e1c9d088c57a8f8bd20'),
 'geometry': {'coordinates': [-58.52671409213821, -34.61362258867436],
              'type': 'Point'},
 'properties': {'AREA_PROG': 'VELEZ SARSFIELD',
                'BARRIO': 'Villa Real',
                'COMUNA': '10',
                'DOMICILIO': '2977 COLIHUE',
                'Dom_geo': 'COLIHUE 2977',
                'Dom_mapa': 'Colihue 2977',
                'ESPECIALID': 'Pediatría, Clínica médica, Tocoginecología, '
                              'Odontología.',
                'NOMBRE': 'CMB N° 19',
                'OBJECTID': 13,
                'OBJETO': 'CENTRO MÉDICO BARRIAL',
                'PISO_DTO': None,
                'REGION_SAN': 'III',
                'TELEFONO': '4566-8314'},
 'type': 'Feature'}
{'_id': ObjectId('63288e1c9d088c57a8f8bd2a'),
 'geometry': {'coordinates': [-58.5040205613833, -34.62799486425087],
              'type': 'Point'},
 'properties': {'AREA_PROG': 'VELEZ SARSFIELD',
                'BARRIO':

In [8]:
#5. Generar a partir de este JSON un archivo de salida de tipo CSV, para su posterior geovisualización en www.gpsvisualizer.com , como fue realizado en el módulo 1, considerando que los campos/encabezados deben ser: latitude, longitude, name, label.
client = MongoClient('mongodb://192.168.0.13:27017/')
filter={}
project={
    'properties.NOMBRE': 1, 
    'properties.Dom_mapa': 1, 
    'geometry.coordinates': 1
}
result = client['salud']['centros'].find(filter=filter, projection=project)
salida = open('centros.csv', 'w', encoding='utf-8')
salida.write('latitude,longitude,name,label\n')
for resultado in result:
   nombre = resultado['properties']['NOMBRE']
   direccion = resultado['properties']['Dom_mapa']
   lon = resultado['geometry']['coordinates'][0]
   lat = resultado['geometry']['coordinates'][1]
   salida.write(f'{lat},{lon},{direccion},{nombre}\n')

salida.close()